<a href="https://colab.research.google.com/github/arlingtunes/Access_Analysis/blob/main/Access_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Access Metrics Made Easy: Nearest Destination and Access Count
This notebook provides a simple workflow and toolkit to calculate find access metrics. Primarily, this notebook calculates the nearest destination and the number of locations within a given threshold. The basic workflow is as follows:
1. Import data: Point data for destinations, origin geographies, and a transit cost matrix of pre-computed travel costs (eg. Minutes, Miles, etc.)
2. Spatially join destinations to origins: Based on the geospatial location, this associates each destination with an origin geography. Given that the travel cost between each origin geography is known, we can easily calculate the distance between.
3. Calculate metrics: For the nearest location, we'll simply sort the list of origins and destinations by travel time, then take the first entry. For the count within a given threshold, we can filter the list of origins and destinations by the travel time, and count the number of entries under a given threshold.

---

Getting start: Imports and a helper function for later. Here, we'll install the needed libraries (on the Colab remote machine) and import the libraries:

In [ ]:
!pip install pandas geopandas access rtree pygeos access
import os
os.environ['USE_PYGEOS'] = '1'
import pandas as pd
import geopandas as gpd
import io
import urllib
from shapely.geometry import Point
# from access import Access, weights, Datasets

def dfToGdf(df, lon, lat, crs='EPSG:4326'):
  '''
    df: pandas dataframe
    lon: longitude column name
    lat: latitude column name
    crs: EPSG code or similar coordinate reference system
  '''
  return gpd.GeoDataFrame(
    df.drop([lon, lat], axis=1),
    crs=crs,
    geometry=[Point(xy) for xy in zip(df[lon], df[lat])])

### Public data:
We've provided a set of public datasets to help get you started. Specifically, we have travel cost matrices for US Zip codes and US Census Tracts, with the travel cost as a value in minutes. The data here is in the Apache Parquet format, an efficient format for storing tabular data.

We've also included some base geographies and population data (should you need it) and some sample destination data of Federally Qualified Health Clinics (FQHC's).

In [58]:
matrices = {
    'tract': {
        'car':'https://uchicago.box.com/shared/static/hkipez75z2p7zivtjdgsfzut4rhm6t6h.parquet',
        'bike':'https://uchicago.box.com/shared/static/cvkq3dytr6rswzrxlgzeejmieq3n5aal.parquet',
        'walk':'https://uchicago.box.com/shared/static/swggh8jxj59c7vpxzx1emt7jnd083rmh.parquet'
    },
    'zip': {
        'car':'https://uchicago.box.com/shared/static/swggh8jxj59c7vpxzx1emt7jnd083rmh.parquet',
        'bike':'https://uchicago.box.com/shared/static/7yzgf1gx3k3sacntjqber6l40m0d5aqw.parquet',
        'walk':'https://uchicago.box.com/shared/static/b3vuqijqys24z146u78dsemn0wvu8i5m.parquet',
    }
}

geographies = {
    'tract': 'https://uchicago.box.com/shared/static/kfoid6fzlbpyfecmwpe9u16cl5n89ru0.zip',
    'zip':'https://uchicago.box.com/shared/static/270ca6syxcg3dlvohhnt3ap92m4t7cxc.zip'
}
pop_data = {
    'tract':'https://uchicago.box.com/shared/static/z6xm6tre935xbc06gg4ukzgyicro26cw.csv',
    'zip': 'https://uchicago.box.com/shared/static/njjpskiuj7amcztrxjws2jfwqlv66t49.csv'
}
sample_point_data = {
    'FQHC': 'https://uchicago.box.com/shared/static/uylcq23g5z8jhvmp7cnofr074j4hwj6e.csv',
    'pharmacies': 'https://uchicago.box.com/shared/static/njjpskiuj7amcztrxjws2jfwqlv66t49.csv',
    'opioid_treatment_facilities': 'https://raw.githubusercontent.com/GeoDaCenter/opioid-policy-scan/master/data_raw/Opioid_Treatment_Directory_Geocoded.csv',
    'moud_full': 'https://raw.githubusercontent.com/GeoDaCenter/opioid-policy-scan/master/data_final/moud/us-wide-moudsCleaned_geocoded.csv',
    'moud1': 'https://raw.githubusercontent.com/spaykin/moud-files/master/small-files/us-wide-moudsCleaned_geocoded1.csv',
    'moud2': 'https://raw.githubusercontent.com/spaykin/moud-files/master/small-files/us-wide-moudsCleaned_geocoded2.csv',
    'moud3': 'https://raw.githubusercontent.com/spaykin/moud-files/master/small-files/us-wide-moudsCleaned_geocoded3.cs',
    'Maryland_2016': 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQeurZkJESe8QvSo0D-L0k4rX2zk2iIAj_hNQtgaWCPKpZL4dKKOLpBeovYr3-3wTtXAHwpIJK7PrMc/pub?gid=707666938&single=true&output=csv'
}

geoid_cols = {
    "tract":"GEOID",
    "zip": "GEOID10"
}

Specify your preferred geographic unit and transit mode below. Or do your own thing!

In [59]:
GEOGRAPHIC_UNIT = 'tract' # 'tract' or 'zip'
TRANSIT_MODE = 'car' # 'car' or 'bike' or 'walk'

TRANSIT_MATRIX = pd.read_parquet(matrices[GEOGRAPHIC_UNIT][TRANSIT_MODE])

url = geographies[GEOGRAPHIC_UNIT]
with urllib.request.urlopen(url) as response:
    with io.BytesIO(response.read()) as zip_file:
        # Write the zip file to disk
        with open("geodata.zip", "wb") as f:
            f.write(zip_file.getbuffer())

# Read the shapefile into a GeoDataFrame
GEOGRAPHIES = gpd.read_file('geodata.zip').to_crs('EPSG:4326')
GEOGRAPHIES['FIPS'] = GEOGRAPHIES[geoid_cols[GEOGRAPHIC_UNIT]].astype('int64')

In [60]:
DESTINATIONS = dfToGdf(pd.read_csv(sample_point_data['Maryland_2016']), 'longitude', 'latitude') # update your destination of interest here (i.e. point locations csv)

In [ ]:
# Green comment, us here: If you're trying to set up some gravity models or other access score calculations,
# you'll probably need population data. We've got you covered. Otherwise ignore this. k? K.
# POPULATION = pd.read_csv(pop_data[GEOGRAPHIC_UNIT])[['FIPS','Total Population']].iloc[1:]
# POPULATION['GEOID'] = POPULATION['FIPS'].astype('int64')
# GEOGRAPHIES = GEOGRAPHIES.merge(POPULATION[['FIPS','Total Population']], how="left", on="FIPS")

### Spatial Join
Spatially joining points and polygons is easy. We're using the `intersects` geometric predicate here, simply meaning that if a point intersects a polygon, those two become joined or associated.

This means that we are able to see which polygon from our geographies each destination is in, and from the travel matrix, we'll know how far it is (roughly) from the other georaphies.

In [61]:
merged_destinations = gpd.sjoin(DESTINATIONS, GEOGRAPHIES[['FIPS', 'geometry']], how='inner', predicate='intersects')
merged_destinations.head()

,ObjectID,Loc_name,Status,Score,Match_type,Match_addr,LongLabel,ShortLabel,Addr_type,Place_addr,...,Xmin,Xmax,Ymin,Ymax,IN_Address,IN_Subregion,IN_Region,geometry,index_right,FIPS
0,1,World,M,100.0,A,"1468 Ellis St, Greenbackville, Virginia, 23356","1468 Ellis St, Greenbackville, VA, 23356, USA",1468 Ellis St,PointAddress,"1468 Ellis St, Greenbackville, Virginia, 23356",...,-75.391340,-75.389340,38.011252,38.013252,1468 Ellis Street,ACCOMACK COUNTY,VA,POINT (-75.39028 38.0124),44959,51001090200
1,2,World,M,100.0,A,"6155 Community Dr, Chincoteague Island, Virgin...","6155 Community Dr, Chincoteague Island, VA, 23...",6155 Community Dr,PointAddress,"6155 Community Dr, Chincoteague Island, Virgin...",...,-75.361425,-75.359425,37.935692,37.937692,6155 Community Dr,ACCOMACK COUNTY,VA,POINT (-75.36061 37.93647),32459,51001090100
2,3,World,M,100.0,A,"10071 Atlantic Rd, Atlantic, Virginia, 23303","10071 Atlantic Rd, Atlantic, VA, 23303, USA",10071 Atlantic Rd,PointAddress,"10071 Atlantic Rd, Atlantic, Virginia, 23303",...,-75.507713,-75.505713,37.899923,37.901923,10071 Atlantic Rd,ACCOMACK COUNTY,VA,POINT (-75.50695 37.90114),44959,51001090200
3,4,World,M,100.0,A,"4264 Firehouse St, New Church, Virginia, 23415","4264 Firehouse St, New Church, VA, 23415, USA",4264 Firehouse St,PointAddress,"4264 Firehouse St, New Church, Virginia, 23415",...,-75.534392,-75.532392,37.977287,37.979287,4264 Firehouse Street,ACCOMACK COUNTY,VA,POINT (-75.53336 37.97853),44959,51001090200
4,5,World,M,100.0,A,"15312 Bayside Dr, Bloxom, Virginia, 23308","15312 Bayside Dr, Bloxom, VA, 23308, USA",15312 Bayside Dr,PointAddress,"15312 Bayside Dr, Bloxom, Virginia, 23308",...,-75.624726,-75.622726,37.828183,37.830183,15312 BAYSIDE DRIVE,ACCOMACK COUNTY,VA,POINT (-75.6235 37.82903),50206,51001090300


### Moins Est Plus
Less is more, let's just snag the columns we need. We'll need to join this data again to the travel matrix, so the second line gets everyone speaking the same language.

In [62]:
## Pull out the simplified columns we need for the analysis
destinations_simplified = merged_destinations[['index_right','FIPS']]
## Type correctly to merge
## Ignore that warning this is fine
destinations_simplified.loc[:,'FIPS'] = destinations_simplified['FIPS'].astype('int64')
destinations_simplified.head()

,index_right,FIPS
0,44959,51001090200
1,32459,51001090100
2,44959,51001090200
3,44959,51001090200
4,50206,51001090300


### The Big Join
Currently, we have destinations associated with our origin geographies (if you're using the default data, census tracts and health clinics), but we need to bring it all together by joining the origins and destinations to the travel matrix. Below, we'll join the travel matrix to the destinations:

In [63]:
## Merge onto the transit matrix, giving us the distance from each origin
## to each destination
merge_transit_matrix = TRANSIT_MATRIX.merge(destinations_simplified, left_on="destination", right_on="FIPS")
merge_transit_matrix.head()

,origin,destination,minutes,index_right,FIPS
0,21121930601,51195931200,103.28,42967,51195931200
1,21121930601,51195931100,111.27,31117,51195931100
2,21121930601,51169030300,133.89,20724,51169030300
3,21121930601,51169030300,133.89,20724,51169030300
4,21121930601,51169030300,133.89,20724,51169030300


### Analysis Time
Let's get down to business. To begin, let's declare some variables that will help us a bit later. To start, we can define what our origin column (by default, creatively, `origin`), the destination ID column that came from the destinations data, the travel cost column, and the treshold for travel time.

In [64]:
origin_col = 'origin'
destination_id_col = 'index_right'
travel_cost_col = 'minutes'
travel_threshold = 30

### Data Cleanup
We have some weird -1000 values. Let's fix them and replace them with 999, the default null value of this travel matrix.

In [65]:
## clean up this weird bug
## then merge the data
merge_transit_matrix = merge_transit_matrix.sort_values(travel_cost_col, ascending=True)
merge_transit_matrix.minutes = merge_transit_matrix.minutes.replace(-1000, 999)
merge_transit_matrix.origin = merge_transit_matrix.origin.astype('int64')
merge_transit_matrix.head()

,origin,destination,minutes,index_right,FIPS
769908,51045050100,51045050100,0.0,27053,51045050100
1171240,51550021502,51550021502,0.0,38427,51550021502
1946044,51051040400,51051040400,0.0,5687,51051040400
1946043,51051040400,51051040400,0.0,5687,51051040400
1405239,51047930502,51047930502,0.0,56343,51047930502


### Nearest location
To get the nearest location, sort the values by lowest cost then filter for  the first appearance of each origin ID. This means we'll get the first time that origin shows up, sorted by the lowest travel cost.

We'll use pandas `.duplicated()` function with the not (`~`) operator before it.

In [66]:
time_to_nearest = merge_transit_matrix[~merge_transit_matrix.origin.duplicated()][[origin_col, travel_cost_col]]
time_to_nearest.head()

,origin,minutes
769908,51045050100,0.0
1171240,51550021502,0.0
1946044,51051040400,0.0
1405239,51047930502,0.0
1065172,51760020600,0.0


### Count in Threshold
For getting the count of destinations within a given threshold (by default, 30 minutes), we can chain a couple functions here from pandas.

First, we'll filter the `travel_costs` dataframe for costs that are less than or equal to the threshold. Then group by the origin column, giving us sets of rows that share the same origin ID, and then count those columns, giving us the number of rows for each origin ID with a travel cost under our treshold.

We'll re-label some columns for easy reference.

In [67]:
## For count, we simple filter for the cost under a given threshold
## Then groupby by and count the results
count_within_threshold = merge_transit_matrix[merge_transit_matrix[travel_cost_col] <= travel_threshold] \
  .groupby(origin_col).count() \
  .reset_index()[[origin_col, travel_cost_col]]
count_within_threshold.columns = [origin_col, f"count within {travel_threshold}"]

### Merge Results
Now, we can merge our two findings into an easy, breezy, beautiful dataframe.

In [68]:
merged_metrics = count_within_threshold.merge(time_to_nearest, on=origin_col, how="outer")
merged_metrics.head()

,origin,count within 30,minutes
0,10001042900,NaN,119.78
1,10001043000,NaN,112.07
2,10001043100,NaN,118.03
3,10001043400,NaN,128.17
4,10005050101,NaN,118.49


### Cleanup
One last edge case to handle here: It is possible some origins are not within 30 minutes of a destination, meaning some of the data will be null. Or, we might have lost some origins from the full geographies dataset.

While not the end of the world, we can clean this up here before shipping of results to our (soon to be disgruntled) data scientist colleagues.

The below finds the missing origin IDs and fills them in, giving us the revered `findings` dataframe.

🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉🎉

In [69]:
## To clean up any missing data, we can check back with our origin list
analyzed_origins = list(merged_metrics[origin_col])
missing_origins = [o for o in GEOGRAPHIES.FIPS if o not in analyzed_origins]

## Then, fill the missing data
missing_data = []
for o in missing_origins:
    missing_entry = {}
    missing_entry[origin_col] = o
    missing_entry[f"count within {travel_threshold}"]=0
    missing_entry[travel_cost_col]=None
    missing_data.append(missing_entry)
missing_df = pd.DataFrame(missing_data)

## and concatenate results
findings = pd.concat([merged_metrics, missing_df])
# Fill any null values with 0 for count within
findings['count within 30'] = findings['count within 30'].fillna(0).astype(int)
# Replace error value "999" in matrices with blanks
findings['minutes'] = findings['minutes'].replace(999.0, None)
findings.head()

<ipython-input-69-ac3376cf5973>:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  findings = pd.concat([merged_metrics, missing_df])


,origin,count within 30,minutes
0,10001042900,0,119.78
1,10001043000,0,112.07
2,10001043100,0,118.03
3,10001043400,0,128.17
4,10005050101,0,118.49


### What's Next?
Well, now you could take this data and export it as a CSV, or join it back to the geographies and visualize it, or try running this analysis with some different data. Ball is in your court, you got this!


In [71]:
# Export to csv
findings.to_csv('Maryland_2016.csv', index = False)

#### Who me?
Oh, sure. If you want to see more, check out [the Covid Atlas](uscovidatlas.org) or [HEROP](https://herop.ssd.uchicago.edu/), [CSDS](https://spatial.uchicago.edu/), or [my github](https://github.com/nofurtherinformation).

Got Q's? Liked this little notebook? Feel free to drop me a line a dhalpern at uchicago dot edu, or on Twitter at [dchalpern](https://twitter.com/dchalpern).